# Practice exercise: Area of applicability 

This is a short practice exercise related to computing the area of applicability of a machine learning model (<a href="https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210X.13650" target="_blank">Meyer and Pebesma, 2021</a>). You have access to a dataset of predictor variables derived from a <a href="https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED" target="_blank">Sentinel-2 satellite image</a> and topographic variables which correspond to points where there are leaf area index (LAI) measurements. The LAI measurements were derived from LiDAR data with a 30 cm spatial resolution and 15 cm vertical resolution averaged to a 10 m spatial resolution to match the size of Sentinel-2 pixels. 

LAI is a measure of the total area of leaves relative to the ground area and is an important biophysical variable for studying vegetation growth and functioning. The data we are using here were collected over the Marburg Forest in Germany and are from the paper by <a href="https://www.sciencedirect.com/science/article/abs/pii/S0304380019303230" target="_blank">Meyer et al. (2019)</a>.

## Setup

### Load data

In [ ]:
import os
import subprocess

if "data-geoml" not in os.listdir(os.getcwd()):
    subprocess.run('wget "https://github.com/envt-5566/geo-ml/raw/main/data/data-geoml.zip"', shell=True, capture_output=True, text=True)
    subprocess.run('unzip "data-geoml.zip"', shell=True, capture_output=True, text=True)
    if "data-geoml.zip" not in os.listdir(os.getcwd()):
        print("Has a directory called data-geoml been downloaded and placed in your working directory? If not, try re-executing this code chunk")
    else:
        print("Data download OK")

DATA_PATH = os.path.join(os.getcwd())

### Load packages

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install xarray[complete]
    !pip install rioxarray
    !pip install mapclassify
    !pip install contextily
    !pip install pysal

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from math import log
from tqdm import tqdm

# plotting
import seaborn as sns
import contextily as cx
import matplotlib.pyplot as plt

# pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# models
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans

In [ ]:
gdf = gpd.read_file(os.path.join(DATA_PATH, "lai_meyer_et_al_2019_marburg.geojson"))
gdf = gdf.drop(columns=["field_1", "ID", "x_utm_25832", "y_utm_25832"])
gdf = gdf.to_crs("EPSG:4326")

## Activity

To compute the area of applicability of a machine learning model we need to i) measure how similar the predictor variables are at a candidate location for prediction to the training data used to develop the model; <a href="https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210X.13650" target="_blank">Meyer and Pebesma (2021)</a> present a dissimilarity index for this task, and ii) determine a threshold dissimilarity index value which delineates the coverage of the training dataset; again <a href="https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210X.13650" target="_blank">Meyer and Pebesma (2021)</a> present a method for this. **Can you adapt the examples from the *area of applicability* notebook to compute the dissimilarity index threshold for the training dataset provided to compute LAI?**

You will need to consider:

* What variables to drop before model training.
* Do you need to fit or train a model? If not, you can pass `None` is as the model argument to `compute_di_threshold()`.
* Should you use random or spatial k-fold cross-validation.

We have copied over the functions from the *area of applicability* notebook to help you. 

### Dissimilarity Index

The dissimilarity index is a measure of the distance between a candidate location for prediction using the machine learning model and the model's training data. It measures distance in predictor variable space and is a measure of how similar the predictor variable values are to the most similar example in the training dataset. The process to compute the dissimilarity index is:

1. Compute the Euclidean distance between the predictor variables for the candidate location and every example in the training dataset.
2. Select the minimum distance.
3. Compute the average Euclidean distance between predictor variables for all examples in the training dataset.
4. Divide the minimum distance between the candidate prediction location and the training dataset (2) by the average distance between predictor variables in the training dataset (3).

The dissimilarity index for the candidate prediction location is the result of step 4. Standardising the dissimilarity index values by dividing by the average distance between predictor variables in the training dataset means that dissimilarity index values range from 0 to infinity. If the dissimilarity index value is 0 then the candidate prediction location is identical to an example in the training dataset.

The following is a helper function that implements steps 2.1, 2.3 and 2.4 in Meyer and Pebesma (2021) and computes the dissimilarity index given arrays of predictor variables for the training dataset and candidate predictor locations.  

In [ ]:
def compute_di(X_train, X_preds, dbar, scaled):
    """Compute the Dissimilarity Index (DI).

    This is based on steps 2.1, 2.3, and 2.4 in Meyer and Pebesma (2021).

    Note, we omit the weighting of predictor variables (step 2.2 of Meyer and Pebesma (2021).
    
    We implement the steps explicitly (i.e. using for loops rather than NumPy broadcasting
    and linear algebra operations). This is for education purposes to support understanding how
    DI is computed. 

    Args:
        X_train (ndarray): Array of training data where examples are aligned on the 0 axis and features on the 1 axis.
        X_preds (ndarray): Array of predictor data where examples are aligned on the 0 axis and features on the 1 axis.
        dbar (float | None): Precomputed mean of dissimilarity index for all values in the training dataset.
        scaled (bool): Flag to indicate if input data is already standardised.

    Returns:
        ndarray, an array of DI values for each example in the prediction set. 
    """
    
    # standardise predictors (using training data)
    if scaled:
        X_train_scaled = X_train
        X_preds_scaled = X_preds
    else:
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_preds_scaled = scaler.transform(X_preds)

    # Compute dissimilarity index (DI)
    # Section 2.3 in Meyer and Pebesma (2021)
    
    # create a matrix to store Euclidean distance between 
    # all training points and prediction points
    n_X_train = X_train_scaled.shape[0]
    n_X_preds = X_preds_scaled.shape[0]

    d_arr = np.zeros((n_X_train, n_X_preds))

    for r in range(0, n_X_train):
        x_train = X_train_scaled[r, :]
        for c in range(0, n_X_preds):
            x_preds = X_preds_scaled[c, :]
            # compute Euclidean distance
            x_dist = np.sqrt(np.sum((x_train - x_preds) * (x_train - x_preds)))
            # store Euclidean distance in d_arr
            d_arr[r, c] = x_dist
            
    # get the minimum distance between each point in x_preds
    # and the training set
    d_arr = np.min(d_arr, axis=0)
    
    # Section 2.4 in Meyer and Pebesma (2021)
    if dbar:
        di = d_arr / dbar
    else:
        # compute average distance between all
        # training points to standardise DI
        d_train_arr = []
        for r in range(0, n_X_train):
            x_train_1 = X_train_scaled[r, :]
            for c in range(r+1, n_X_train):
                x_train_2 = X_train_scaled[c, :]
                # compute Euclidean distance
                x_dist = np.sqrt(np.sum((x_train_1 - x_train_2) * (x_train_1 - x_train_2)))
                # store Euclidean distance in d_arr
                d_train_arr.append(x_dist)
    
        di = d_arr / np.mean(d_train_arr)

    return di

In [ ]:
def compute_dbar(X_train, scaled):
    """Compute average DI between all values in the training dataset.
    
    d_bar is computed section 2.4 in Meyer and Pebesma (2021). It is 
    used to standardise dissimilarity index values. 

    Precomputing can save time when the training dataset is fixed and we
    want to iterate over a range of candidate prediction locations.

    We implement the steps explicitly (i.e. using for loops rather than NumPy broadcasting
    and linear algebra operations). This is for education purposes to support understanding how
    dbar is computed. 
    """
    # standardise predictors (using training data)
    if scaled:
        X_train_scaled = X_train
    else:
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)

    n_X_train = X_train.shape[0]
        
    # Section 2.4 in Meyer and Pebesma (2021)
    
    # compute average distance between all
    # training points to standardise DI
    d_train_arr = []
    for r in range(0, n_X_train):
        x_train_1 = X_train_scaled[r, :]
        for c in range(r+1, n_X_train):
            x_train_2 = X_train_scaled[c, :]
            # compute Euclidean distance
            x_dist = np.sqrt(np.sum((x_train_1 - x_train_2) * (x_train_1 - x_train_2)))
            # store Euclidean distance in d_arr
            d_train_arr.append(x_dist)
            
    return np.mean(d_train_arr)

### Dissimilarity index threshold

Meyer and Pebesma (2021) in section 2.5 compute a dissimilarity index threshold which represents the outlier removed maximum dissimilarity of the training dataset. This represents a threshold for the model's area of applicability. If a candidate prediction location has a dissimilarity index greater than this threshold it is deemed outside the model's area of applicability. 

The dissimilarity index threshold is computed using k-fold cross-validation on the training dataset:

1. Split the training dataset into $k$ folds.
2. Hold out each $k$ fold in-turn:
    * Compute the dissimilarity index between all values in the hold-out fold and training folds.
    * Optional step: Compute model predictions and error for all values in the hold-out fold.
3. Compute the outlier-removed threshold as the 75th percentile dissimilarity index value + (1.5 * IQR)

The following helper function computes the dissimilarity index threshold given the training data as a `GeoDataFrame`. 

In [ ]:
def compute_di_theshold(gdf, n_folds, model, spatial_cv, target_column, cols_to_drop):
    """
    Function to compute DI threshold following Meyer and Pebesma (2021).

    The DI threshold is computed as the 75th percentile + (1.5 * IQR). DI threshold is 
    considered the outlier removed maximum DI of the training data set. Data points with a 
    DI greater than the DI threshold are outside the area of applicability of the model
    given the training data. 

    We implement the steps explicitly (i.e. using for loops rather than NumPy broadcasting
    and linear algebra operations). This is for education purposes to support understanding how
    the DI threshold is computed. 
    
    Args:
        gdf (GeoDataFrame): The spatial dataset (GeoDataFrame) containing features and the target variable.
        n_folds (int): The number of folds to use for cross-validation.
        model: The machine learning model to train and evaluate.
        spatial_cv (bool): If `True`, use k-fold spatial cross-validation. Otherwise, use k-fold random cross-validation.
        target_column (str): The column name for the target variable.
        cols_to_drop (list): Columns to drop that should not be included as predictors.

    Returns:
        dict, dict object with the DI threshold, the DI values for the hold out folds, the errors
        for the hold out folds, and the predictions for the hold out folds.
    
    """
    # Section 2.5 in Meyer and Pebesma (2021)
    if spatial_cv:
        gdf_tmp = gdf.copy()

        # split the data into n_folds spatially
        # assign a fold id column to each row
    
        # add x and y columns
        gdf_tmp.loc[:, "x"] = gdf_tmp.loc[:, "geometry"].x
        gdf_tmp.loc[:, "y"] = gdf_tmp.loc[:, "geometry"].y
    
        # create an array of x and y values
        X = gdf_tmp.loc[:, ["x", "y"]].copy()
    
        # create a KMeans clusterer
        km = KMeans(
            n_clusters=n_folds,
            init="random",
            n_init=10,
            max_iter=300,
            tol=1e-04,
            random_state=123,
        )
        spatial_folds = km.fit_predict(X)
        gdf_tmp.loc[:, "fold"] = spatial_folds
    
        # keep a copy of GeoDataFrame with folds for plotting
        gdf_spatial_folds = gdf_tmp.copy()

        # drop columns that should not be included as predictors
        gdf_tmp = gdf_tmp.drop(columns=cols_to_drop)
        
    else:
        gdf_tmp = gdf.copy()

        # split the data into n_folds
        # assign a fold id column to each row
        # shuffle the GeoDataFrame
        gdf_tmp = gdf_tmp.sample(frac=1, random_state=123)
        
        # assign a fold id using the modulo operation
        gdf_tmp.loc[:, "fold"] = np.arange(len(gdf_tmp)) % n_folds
        
        # store shuffled gdf to return for plotting
        gdf_folds = gdf_tmp.copy()

        # drop columns that should not be included as predictors
        gdf_tmp = gdf_tmp.drop(columns=cols_to_drop)

    # list to store DI's computed on the hold out folds
    dis = []

    # predictions on cv hold out folds
    predictions = []
    
    # errors on cv hold out folds
    errors = []
    
    # loop over folds and compute DI
    for k in range(0, n_folds):
        # subset out preds fold
        gdf_preds_fold = gdf_tmp.loc[gdf_tmp["fold"] == k, :]
        
        # subset out training folds
        # != is the not equals to operator
        gdf_train_fold = gdf_tmp.loc[gdf_tmp["fold"] != k, :]

        # subset training folds into predictors and targets arrays
        X_train = gdf_train_fold.drop(columns=[target_column, "fold"])
        y_train = gdf_train_fold.loc[:, target_column]

        # subset preds fold into predictors and targets arrays
        X_preds = gdf_preds_fold.drop(columns=[target_column, "fold"])
        y_preds = gdf_preds_fold.loc[:, target_column]

        # standardise data
        tmp_scaler = StandardScaler().fit(X_train)
        X_train_scaled = tmp_scaler.transform(X_train)
        X_preds_scaled = tmp_scaler.transform(X_preds)

        # use compute_df function previously defined
        fold_dis = compute_di(X_train_scaled, X_preds_scaled, None, scaled=True)

        dis = dis + fold_dis.tolist()

        # train model 
        if model:
            m = model.fit(X_train_scaled, y_train)
    
            # compute predictions and error using the hold out fold 
            y_test_preds = m.predict(X_preds_scaled)
            predictions = predictions + y_test_preds.tolist()
    
            y_preds_errors = y_test_preds - y_preds
            errors = errors + y_preds_errors.tolist()

    # compute DI threshold
    # 75th percentile 1.5 * IQR as in Meyer and Pebesma (2021)
    iqr = np.percentile(dis, 75) - np.percentile(dis, 25)
    di_threshold = np.percentile(dis, 75) + (1.5 * iqr)

    output = {}
    output["di_threshold"] = di_threshold
    output["di"] = dis
    output["targets"] = y_preds
    output["predictions"] = predictions
    output["errors"] = errors

    return output